In [3]:
import numpy as np
from scipy.optimize import minimize_scalar, brentq
from math import exp, sqrt, log
import warnings

In [ ]:
def continuous_to_apr(r):
    """
    Converts a continuously compounded annual rate to an Annual Percentage Rate (APR).
    :param r: Continuous annual rate (as a decimal)
    :return: APR (as a decimal)
    """
    return exp(r) - 1

def apr_to_continuous(apr):
    """
    Converts an APR to the equivalent continuous rate.
    :param apr: Annual Percentage Rate (as a decimal)
    :return: Continuous rate (as a decimal)
    """
    return log(1 + apr) 

Start from:

Protective Put = long put + long stock, $payoff = S_T + max(0, K-S_T)$, $profit = payoff -(S+P)e^{rt}$

Put spread = long put (K2) + short put (K1) + long stock, $payoff = S_T - max(0,K_1-S_T) + max(0,K_2-S_T)$, profit = payoff - future value of the net cost

Collars = long put (K1) + short call (K2) + long stock (when we want a complete floor like protective put, but do not want to cost that much), $payoff = S_T + max(0,K_1-S_T) - max(0, S_T-K_2)$, $profit = payoff - (S_0+P_{K1} - C_{K2})e^{rt}S_T$

3-way collar, sum of four positions:
- Sell put at K1
- Buy put at K2
- Sell call at K3
- Long stock

$payoff = S_T - max(0-K_0-S_T) + max(0,K_1-S_T) - max(0, S_T-K_2)$


Paylater = 
- Buying 2 puts at K1
- Selling 1 put at K2
- Long stock

$payoff = S_T + 2*max(0,K_1-S_T) - max(0,K_2-S_T)$


In [5]:
def protective_put(S_T, K, S_0, P, r, t):
    """
    Calculate protective put strategy payoff and profit
    Strategy: Long put + Long stock (simple downside protection)
    
    Parameters:
    S_T (float): Stock price at expiration - final stock price when put expires
                 Example: S_T=85 means stock ends at $85
    K (float): Strike price of protective put - your downside protection level
               Example: K=90 means you're protected below $90
    S_0 (float): Initial stock price when strategy was established
                 Example: S_0=100 means you bought stock at $100
    P (float): Put option premium paid - cost of downside protection
               Example: P=4.5 means you paid $4.50 per share for put protection
    r (float): Risk-free interest rate (annual, as decimal)
               Example: r=0.05 means 5% annual rate
    t (float): Time to expiration (in years)
               Example: t=0.25 means 3 months
    
    Returns:
    tuple: (payoff, profit) where:
    - payoff: Total value at expiration (stock value + put payoff)
    - profit: Net profit/loss accounting for initial costs and interest
    
    Strategy Profile:
    - Above K: You keep stock gains, put expires worthless
    - Below K: Put protects you - minimum value is K regardless of how low stock goes
    """
    payoff = S_T + max(0, K - S_T)
    profit = payoff - (S_0 + P) * exp(r * t)
    return payoff, profit

def put_spread(S_T, K1, K2, S_0, P1, P2, r, t):
    """
    Calculate put spread strategy payoff and profit
    Strategy: Long put (K2) + Short put (K1) + Long stock (partial protection)
    
    Parameters:
    S_T (float): Stock price at expiration - final stock price when options expire
                 Example: S_T=82 means stock ends at $82
    K1 (float): Strike price of SHORT put (lower strike) - where you SELL protection
                Example: K1=80 means you sell puts at $80 (take risk below $80)
    K2 (float): Strike price of LONG put (higher strike) - where you BUY protection
                Example: K2=95 means you buy puts at $95 (protected below $95)
                Note: K2 > K1, you're protected between K1 and K2
    S_0 (float): Initial stock price when strategy was established
                 Example: S_0=100 means original stock purchase price
    P1 (float): Premium RECEIVED from selling the put at K1
                Example: P1=2.3 means you received $2.30 for selling put
    P2 (float): Premium PAID for buying the put at K2
                Example: P2=6.8 means you paid $6.80 for protective put
    r (float): Risk-free interest rate (annual, as decimal)
               Example: r=0.04 means 4% annual rate
    t (float): Time to expiration (in years)
               Example: t=0.5 means 6 months
    
    Returns:
    tuple: (payoff, profit) where:
    - payoff: Total portfolio value at expiration
    - profit: Net profit/loss after accounting for all premiums and interest
    
    Strategy Profile:
    - Above K2: Stock gains only, both puts expire worthless
    - K1 to K2: Protected zone - you're hedged in this range
    - Below K1: You lose on short put, protection limited to spread width (K2-K1)
    """
    payoff = S_T - max(0, K1 - S_T) + max(0, K2 - S_T)
    net_cost = S_0 + P2 - P1  # Stock + Long put premium - Short put premium
    profit = payoff - net_cost * exp(r * t)
    return payoff, profit

def paylater(S_T, K1, K2, S_0, P1, P2, r, t):
    """
    Calculate paylater strategy payoff and profit
    Strategy: Buy 2 puts at K1 + Sell 1 put at K2 + Long stock (leveraged protection)
    
    Parameters:
    S_T (float): Stock price at expiration - final stock price when options expire
                 Example: S_T=78 means stock ends at $78
    K1 (float): Strike price of LONG puts (where you buy 2 contracts)
                Example: K1=85 means you buy 2 puts at $85 strike
                This provides 2x protection below this level
    K2 (float): Strike price of SHORT put (where you sell 1 contract)
                Example: K2=95 means you sell 1 put at $95 strike
                Note: Typically K2 > K1 for financing the strategy
    S_0 (float): Initial stock price when strategy was established
                 Example: S_0=100 means original stock purchase price
    P1 (float): Premium PAID for each long put at K1
                Example: P1=3.2 means you paid $3.20 per share for each put
                Total cost for 2 puts = 2 * P1
    P2 (float): Premium RECEIVED from selling the put at K2
                Example: P2=7.5 means you received $7.50 for selling the put
    r (float): Risk-free interest rate (annual, as decimal)
               Example: r=0.03 means 3% annual rate
    t (float): Time to expiration (in years)
               Example: t=0.33 means 4 months
    
    Returns:
    tuple: (payoff, profit) where:
    - payoff: Total portfolio value at expiration
    - profit: Net profit/loss after accounting for all premiums and interest
    
    Strategy Profile:
    - Above K2: Stock gains only, all puts expire worthless
    - K1 to K2: 2x put protection from long puts, short put expires worthless
    - Below K1: 2x protection continues, but you also lose on short put at K2
    - This creates leveraged downside protection but with more complex risk below K1
    """
    payoff = S_T + 2 * max(0, K1 - S_T) - max(0, K2 - S_T)
    net_cost = S_0 + 2 * P1 - P2  # Stock + 2 long puts - 1 short put
    profit = payoff - net_cost * exp(r * t)
    return payoff, profit

def collar(S_T, K1, K2, S_0, P, C, r, t):
    """
    Calculate collar strategy payoff and profit
    Strategy: Long put (K1) + Short call (K2) + Long stock
    
    Parameters:
    S_T (float): Stock price at expiration - the final stock price when options expire
                 Example: S_T=95 means stock is at $95 at expiration
    K1 (float): Strike price of protective put (floor) - your downside protection level
                Example: K1=90 means you're protected below $90
    K2 (float): Strike price of covered call (ceiling) - your upside cap level  
                Example: K2=110 means you give up gains above $110
    S_0 (float): Initial stock price when strategy was established
                 Example: S_0=100 means you bought stock at $100
    P (float): Put option premium paid - cost of downside protection
               Example: P=3.5 means you paid $3.50 per share for put
    C (float): Call option premium received - income from selling upside
               Example: C=2.0 means you received $2.00 per share for call
    r (float): Risk-free interest rate (annual, as decimal)
               Example: r=0.05 means 5% annual rate
    t (float): Time to expiration (in years)
               Example: t=0.25 means 3 months
    
    Returns:
    tuple: (payoff, profit) where:
    - payoff: Total value at expiration (stock + option payoffs)  
    - profit: Net profit/loss accounting for initial costs and interest
    """
    payoff = S_T + max(0, K1 - S_T) - max(0, S_T - K2)
    net_cost = S_0 + P - C
    profit = payoff - net_cost * exp(r * t)
    return payoff, profit

def three_way_collar(S_T, K1, K2, K3, S_0, P1, P2, C, r, t):
    """
    Calculate 3-way collar strategy payoff and profit
    Strategy: Sell put at K1 + Buy put at K2 + Sell call at K3 + Long stock
    
    Parameters:
    S_T (float): Stock price at expiration - final stock price when options expire
                 Example: S_T=88 means stock ends at $88
    K1 (float): Strike price of SHORT put (lowest strike) - where you SELL protection
                Example: K1=80 means you sell puts at $80 (take risk below $80)
    K2 (float): Strike price of LONG put (middle strike) - where you BUY protection
                Example: K2=90 means you buy puts at $90 (protected below $90)  
                Note: K2 > K1, you're protected between K1 and K2
    K3 (float): Strike price of SHORT call (highest strike) - where you SELL calls
                Example: K3=115 means you sell calls at $115 (cap gains above $115)
                Note: K3 > K2 > K1
    S_0 (float): Initial stock price when strategy was established
                 Example: S_0=100 means original stock purchase price
    P1 (float): Premium RECEIVED from selling the put at K1
                Example: P1=1.8 means you received $1.80 for selling put
    P2 (float): Premium PAID for buying the put at K2  
                Example: P2=4.2 means you paid $4.20 for protective put
    C (float): Premium RECEIVED from selling the call at K3
               Example: C=2.5 means you received $2.50 for selling call
    r (float): Risk-free interest rate (annual, as decimal)
               Example: r=0.04 means 4% annual rate
    t (float): Time to expiration (in years)
               Example: t=0.5 means 6 months
    
    Returns:
    tuple: (payoff, profit) where:
    - payoff: Total portfolio value at expiration
    - profit: Net profit/loss after accounting for all premiums and interest
    
    Payoff Profile:
    - Below K1: You lose on short put, but protected by long put above K2
    - K1 to K2: Protected zone - limited downside  
    - K2 to K3: Stock gains/losses only
    - Above K3: Gains capped by short call
    """
    payoff = (S_T 
              - max(0, K1 - S_T)  # Short put at K1
              + max(0, K2 - S_T)  # Long put at K2
              - max(0, S_T - K3)) # Short call at K3
    
    net_cost = S_0 + P2 - P1 - C
    profit = payoff - net_cost * exp(r * t)
    return payoff, profit

def black_scholes_call(S, K, r, T, sigma):
    """Calculate Black-Scholes call option price"""
    from scipy.stats import norm
    
    if T <= 0:
        return max(0, S - K)
    
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    call_price = S*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
    return call_price

def black_scholes_put(S, K, r, T, sigma):
    """Calculate Black-Scholes put option price"""
    from scipy.stats import norm
    
    if T <= 0:
        return max(0, K - S)
    
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    put_price = K*exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    return put_price

In [6]:
class HedgingCompleter:
    def __init__(self, S_0, r, t, sigma):
        """
        Initialize the hedging completer with market parameters
        
        Parameters:
        S_0 (float): Current stock price (spot price)
                     Example: S_0=100 means stock currently trades at $100
        r (float): Risk-free interest rate (annual, as decimal)
                   Example: r=0.05 means 5% annual risk-free rate
                   Used for: discounting, Black-Scholes calculations
        t (float): Time to expiration (in years, as decimal)
                   Example: t=0.25 means 3 months (quarter year)
                   t=0.083 means 1 month, t=1.0 means 1 year
        sigma (float): Implied volatility of the underlying stock (annual, as decimal)
                      Example: sigma=0.25 means 25% annual volatility
                      Used for: Black-Scholes option pricing
                      Higher sigma = more expensive options
        """
        self.S_0 = S_0
        self.r = r
        self.t = t
        self.sigma = sigma
    
    def get_option_price(self, K, option_type='call'):
        """Get option price using Black-Scholes"""
        if option_type == 'call':
            return black_scholes_call(self.S_0, K, self.r, self.t, self.sigma)
        else:
            return black_scholes_put(self.S_0, K, self.r, self.t, self.sigma)
        
    # PROTECTIVE PUT COMPLETION METHODS
    def complete_protective_put_given_strike(self, K, target_cost=None, target_payoff=None):
        """
        Given put strike, calculate required stock position for target cost or payoff
        
        Parameters:
        K (float): Strike price of protective put - your downside protection level
                   Example: K=90 means you're protected below $90
        target_cost (float, optional): Desired total upfront cost of the strategy
                                      Example: target_cost=50000 means willing to spend $50k total
        target_payoff (float, optional): Desired minimum payoff at expiration
                                        Example: target_payoff=45000 means want at least $45k value
        
        Note: Provide either target_cost OR target_payoff, not both
        
        Returns:
        Dictionary with required stock shares and strategy details
        """
        P = self.get_option_price(K, 'put')
        
        if target_cost is not None:
            # target_cost = n_shares * (S_0 + P)
            n_shares = target_cost / (self.S_0 + P)
            
            return {
                'strategy': 'protective_put',
                'stock_shares': n_shares,
                'put_strike': K,
                'put_premium': P,
                'stock_cost': n_shares * self.S_0,
                'put_cost': n_shares * P,
                'total_cost': target_cost,
                'minimum_payoff': n_shares * K,
                'breakeven_price': self.S_0 + P
            }
        
        elif target_payoff is not None:
            # For protective put, minimum payoff = n_shares * K
            n_shares = target_payoff / K
            
            return {
                'strategy': 'protective_put',
                'stock_shares': n_shares,
                'put_strike': K,
                'put_premium': P,
                'stock_cost': n_shares * self.S_0,
                'put_cost': n_shares * P,
                'total_cost': n_shares * (self.S_0 + P),
                'minimum_payoff': target_payoff,
                'breakeven_price': self.S_0 + P
            }
        
        else:
            raise ValueError("Must provide either target_cost or target_payoff")
    
    def complete_protective_put_find_strike(self, stock_shares=1, target_cost=None, min_protection_level=None):
        """
        Given stock position, find put strike for target cost or protection level
        
        Parameters:
        stock_shares (float): Number of stock shares to protect
                             Default: 1 share
                             Example: stock_shares=1000 for 1,000 shares
        target_cost (float, optional): Desired total cost of the strategy
                                      Example: target_cost=52000 for $52k total cost
        min_protection_level (float, optional): Minimum acceptable protection level (as % of stock price)
                                              Example: min_protection_level=0.9 for 90% protection
        
        Returns:
        Dictionary with optimal put strike and strategy details
        """
        if target_cost is not None:
            # target_cost = stock_shares * (S_0 + P)
            required_put_premium = (target_cost / stock_shares) - self.S_0
            
            def objective(K):
                P = self.get_option_price(K, 'put')
                return (P - required_put_premium)**2
            
            try:
                result = minimize_scalar(objective, bounds=(self.S_0*0.5, self.S_0*1.2), method='bounded')
                K_optimal = result.x
                P_optimal = self.get_option_price(K_optimal, 'put')
                
                return {
                    'strategy': 'protective_put',
                    'stock_shares': stock_shares,
                    'put_strike': K_optimal,
                    'put_premium': P_optimal,
                    'total_cost': target_cost,
                    'protection_level': K_optimal / self.S_0,
                    'optimization_success': result.success
                }
            except:
                return {'error': 'Could not find suitable put strike for target cost'}
        
        elif min_protection_level is not None:
            K_optimal = self.S_0 * min_protection_level
            P_optimal = self.get_option_price(K_optimal, 'put')
            
            return {
                'strategy': 'protective_put',
                'stock_shares': stock_shares,
                'put_strike': K_optimal,
                'put_premium': P_optimal,
                'total_cost': stock_shares * (self.S_0 + P_optimal),
                'protection_level': min_protection_level,
                'minimum_payoff': stock_shares * K_optimal
            }
        
        else:
            raise ValueError("Must provide either target_cost or min_protection_level")
    
    # PUT SPREAD COMPLETION METHODS
    def complete_put_spread_given_strikes(self, K1, K2, target_cost=None, target_payoff=None):
        """
        Given both put strikes, calculate required stock position
        
        Parameters:
        K1 (float): Strike price of SHORT put (lower strike) - where you SELL protection
                    Example: K1=85 means you sell puts at $85 (take risk below $85)
        K2 (float): Strike price of LONG put (higher strike) - where you BUY protection
                    Example: K2=95 means you buy puts at $95 (protected below $95)
                    Note: K2 > K1
        target_cost (float, optional): Desired total upfront cost
        target_payoff (float, optional): Desired payoff in the protected zone
        
        Returns:
        Dictionary with required stock shares and strategy details
        """
        if K1 >= K2:
            raise ValueError("K1 (short put strike) must be less than K2 (long put strike)")
        
        P1 = self.get_option_price(K1, 'put')  # Short put premium (received)
        P2 = self.get_option_price(K2, 'put')  # Long put premium (paid)
        
        if target_cost is not None:
            # target_cost = n_shares * (S_0 + P2 - P1)
            net_cost_per_share = self.S_0 + P2 - P1
            n_shares = target_cost / net_cost_per_share
            
            return {
                'strategy': 'put_spread',
                'stock_shares': n_shares,
                'short_put_strike': K1,
                'long_put_strike': K2,
                'short_put_premium': P1,
                'long_put_premium': P2,
                'net_cost_per_share': net_cost_per_share,
                'total_cost': target_cost,
                'max_protection': n_shares * K2,
                'min_protection': n_shares * K1,
                'protection_width': K2 - K1
            }
        
        elif target_payoff is not None:
            # In protected zone (K1 < S_T < K2), payoff = n_shares * K2
            n_shares = target_payoff / K2
            net_cost = n_shares * (self.S_0 + P2 - P1)
            
            return {
                'strategy': 'put_spread',
                'stock_shares': n_shares,
                'short_put_strike': K1,
                'long_put_strike': K2,
                'short_put_premium': P1,
                'long_put_premium': P2,
                'total_cost': net_cost,
                'protected_payoff': target_payoff,
                'max_loss_below_K1': target_payoff - n_shares * (K2 - K1)
            }
        
        else:
            raise ValueError("Must provide either target_cost or target_payoff")
    
    def complete_put_spread_find_long_put(self, K1, stock_shares=1, target_cost=None):
        """
        Given short put strike and stock position, find long put strike
        
        Parameters:
        K1 (float): Strike price of SHORT put (lower strike)
                    Example: K1=80 means you sell puts at $80
        stock_shares (float): Number of stock shares in position
        target_cost (float): Desired total cost of strategy
        
        Returns:
        Dictionary with optimal long put strike and strategy details
        """
        P1 = self.get_option_price(K1, 'put')
        
        if target_cost is not None:
            # target_cost = stock_shares * (S_0 + P2 - P1)
            required_long_put_premium = (target_cost / stock_shares) - self.S_0 + P1
            
            def objective(K2):
                if K2 <= K1:
                    return float('inf')
                P2 = self.get_option_price(K2, 'put')
                return (P2 - required_long_put_premium)**2
            
            try:
                result = minimize_scalar(objective, bounds=(K1*1.01, self.S_0*1.2), method='bounded')
                K2_optimal = result.x
                P2_optimal = self.get_option_price(K2_optimal, 'put')
                
                return {
                    'strategy': 'put_spread',
                    'stock_shares': stock_shares,
                    'short_put_strike': K1,
                    'long_put_strike': K2_optimal,
                    'short_put_premium': P1,
                    'long_put_premium': P2_optimal,
                    'total_cost': target_cost,
                    'protection_width': K2_optimal - K1,
                    'optimization_success': result.success
                }
            except:
                return {'error': 'Could not find suitable long put strike'}
        
        else:
            raise ValueError("Must provide target_cost")
    
    def complete_put_spread_find_short_put(self, K2, stock_shares=1, target_cost=None):
        """
        Given long put strike and stock position, find short put strike
        
        Parameters:
        K2 (float): Strike price of LONG put (higher strike)
                    Example: K2=95 means you buy puts at $95
        stock_shares (float): Number of stock shares in position
        target_cost (float): Desired total cost of strategy
        
        Returns:
        Dictionary with optimal short put strike and strategy details
        """
        P2 = self.get_option_price(K2, 'put')
        
        if target_cost is not None:
            # target_cost = stock_shares * (S_0 + P2 - P1)
            required_short_put_premium = self.S_0 + P2 - (target_cost / stock_shares)
            
            def objective(K1):
                if K1 >= K2:
                    return float('inf')
                P1 = self.get_option_price(K1, 'put')
                return (P1 - required_short_put_premium)**2
            
            try:
                result = minimize_scalar(objective, bounds=(self.S_0*0.3, K2*0.99), method='bounded')
                K1_optimal = result.x
                P1_optimal = self.get_option_price(K1_optimal, 'put')
                
                return {
                    'strategy': 'put_spread',
                    'stock_shares': stock_shares,
                    'short_put_strike': K1_optimal,
                    'long_put_strike': K2,
                    'short_put_premium': P1_optimal,
                    'long_put_premium': P2,
                    'total_cost': target_cost,
                    'protection_width': K2 - K1_optimal,
                    'optimization_success': result.success
                }
            except:
                return {'error': 'Could not find suitable short put strike'}
        
        else:
            raise ValueError("Must provide target_cost")
    
    # PAYLATER COMPLETION METHODS
    def complete_paylater_given_strikes(self, K1, K2, target_cost=None, target_payoff=None):
        """
        Given both strikes, calculate required stock position for paylater strategy
        
        Parameters:
        K1 (float): Strike price of LONG puts (buy 2 contracts)
                    Example: K1=85 means you buy 2 puts at $85
        K2 (float): Strike price of SHORT put (sell 1 contract)
                    Example: K2=95 means you sell 1 put at $95
        target_cost (float, optional): Desired total upfront cost
        target_payoff (float, optional): Desired payoff in protection zone
        
        Returns:
        Dictionary with required stock shares and strategy details
        """
        P1 = self.get_option_price(K1, 'put')  # Long put premium (paid, buy 2)
        P2 = self.get_option_price(K2, 'put')  # Short put premium (received, sell 1)
        
        if target_cost is not None:
            # target_cost = n_shares * (S_0 + 2*P1 - P2)
            net_cost_per_share = self.S_0 + 2*P1 - P2
            n_shares = target_cost / net_cost_per_share
            
            return {
                'strategy': 'paylater',
                'stock_shares': n_shares,
                'long_put_strike': K1,
                'short_put_strike': K2,
                'long_put_premium': P1,
                'short_put_premium': P2,
                'long_puts_quantity': 2 * n_shares,
                'short_puts_quantity': n_shares,
                'net_cost_per_share': net_cost_per_share,
                'total_cost': target_cost,
                'leveraged_protection': True
            }
        
        elif target_payoff is not None:
            # Complex payoff structure - need to specify at what price level
            # Assuming target_payoff at K1 level where max protection kicks in
            # At K1: payoff = n_shares * (K1 + 2*(K1-K1) - max(0, K2-K1)) = n_shares * (K1 - max(0, K2-K1))
            if K2 > K1:
                payoff_at_K1 = K1 - (K2 - K1)  # K1 - (K2 - K1) = 2*K1 - K2
            else:
                payoff_at_K1 = K1
            
            n_shares = target_payoff / payoff_at_K1
            net_cost = n_shares * (self.S_0 + 2*P1 - P2)
            
            return {
                'strategy': 'paylater',
                'stock_shares': n_shares,
                'long_put_strike': K1,
                'short_put_strike': K2,
                'long_put_premium': P1,
                'short_put_premium': P2,
                'total_cost': net_cost,
                'target_payoff': target_payoff,
                'payoff_at_K1': target_payoff
            }
        
        else:
            raise ValueError("Must provide either target_cost or target_payoff")
    
    def complete_paylater_find_short_put(self, K1, stock_shares=1, target_cost=None):
        """
        Given long put strike and stock position, find short put strike for target cost
        
        Parameters:
        K1 (float): Strike price of LONG puts (buy 2 contracts)
                    Example: K1=80 means you buy 2 puts at $80
        stock_shares (float): Number of stock shares in position
        target_cost (float): Desired total cost of strategy
        
        Returns:
        Dictionary with optimal short put strike and strategy details
        """
        P1 = self.get_option_price(K1, 'put')
        
        if target_cost is not None:
            # target_cost = stock_shares * (S_0 + 2*P1 - P2)
            required_short_put_premium = self.S_0 + 2*P1 - (target_cost / stock_shares)
            
            def objective(K2):
                P2 = self.get_option_price(K2, 'put')
                return (P2 - required_short_put_premium)**2
            
            try:
                result = minimize_scalar(objective, bounds=(self.S_0*0.5, self.S_0*1.5), method='bounded')
                K2_optimal = result.x
                P2_optimal = self.get_option_price(K2_optimal, 'put')
                
                return {
                    'strategy': 'paylater',
                    'stock_shares': stock_shares,
                    'long_put_strike': K1,
                    'short_put_strike': K2_optimal,
                    'long_put_premium': P1,
                    'short_put_premium': P2_optimal,
                    'total_cost': target_cost,
                    'optimization_success': result.success,
                    'net_financing': P2_optimal - 2*P1
                }
            except:
                return {'error': 'Could not find suitable short put strike'}
        
        else:
            raise ValueError("Must provide target_cost")
    
    def complete_paylater_find_long_put(self, K2, stock_shares=1, target_cost=None):
        """
        Given short put strike and stock position, find long put strike for target cost
        
        Parameters:
        K2 (float): Strike price of SHORT put (sell 1 contract)
                    Example: K2=95 means you sell 1 put at $95
        stock_shares (float): Number of stock shares in position
        target_cost (float): Desired total cost of strategy
        
        Returns:
        Dictionary with optimal long put strike and strategy details
        """
        P2 = self.get_option_price(K2, 'put')
        
        if target_cost is not None:
            # target_cost = stock_shares * (S_0 + 2*P1 - P2)
            required_long_put_premium = ((target_cost / stock_shares) - self.S_0 + P2) / 2
            
            def objective(K1):
                P1 = self.get_option_price(K1, 'put')
                return (P1 - required_long_put_premium)**2
            
            try:
                result = minimize_scalar(objective, bounds=(self.S_0*0.3, self.S_0*1.2), method='bounded')
                K1_optimal = result.x
                P1_optimal = self.get_option_price(K1_optimal, 'put')
                
                return {
                    'strategy': 'paylater',
                    'stock_shares': stock_shares,
                    'long_put_strike': K1_optimal,
                    'short_put_strike': K2,
                    'long_put_premium': P1_optimal,
                    'short_put_premium': P2,
                    'total_cost': target_cost,
                    'optimization_success': result.success,
                    'leverage_ratio': 2.0  # 2 long puts per 1 short put
                }
            except:
                return {'error': 'Could not find suitable long put strike'}
        
        else:
            raise ValueError("Must provide target_cost")
    # COLLAR COMPLETION METHODS
    def complete_collar_given_strikes(self, K1, K2, target_payoff, S_T_scenarios=None):
        """
        Given put strike (K1) and call strike (K2), calculate required stock position
        
        Parameters:
        K1 (float): Put strike price - the floor price below which you're protected
                    Example: K1=95 means you're protected if stock falls below $95
        K2 (float): Call strike price - the ceiling price above which gains are capped
                    Example: K2=105 means you give up gains above $105
        target_payoff (float): Desired total payoff value at expiration
                              Example: target_payoff=100000 means you want $100k total value
        S_T_scenarios (list, optional): List of stock prices to test the strategy against
                                       Default: [80% of current, current, 120% of current price]
                                       Example: [80, 100, 120] for different market scenarios
        
        Returns:
        Dictionary with stock shares needed and hedge details
        """
        if S_T_scenarios is None:
            S_T_scenarios = [self.S_0 * 0.8, self.S_0, self.S_0 * 1.2]
        
        P = self.get_option_price(K1, 'put')
        C = self.get_option_price(K2, 'call')
        
        # For collar: payoff = n_shares * S_T + n_shares * max(0, K1 - S_T) - n_shares * max(0, S_T - K2)
        # where n_shares is the number of stock shares
        
        # Try to solve for n_shares using different scenarios
        n_shares_estimates = []
        
        for S_T in S_T_scenarios:
            option_payoff = max(0, K1 - S_T) - max(0, S_T - K2)
            total_stock_payoff = S_T + option_payoff
            
            if abs(total_stock_payoff) > 1e-10:  # Avoid division by zero
                n_shares = target_payoff / total_stock_payoff
                n_shares_estimates.append(n_shares)
        
        # Use average of estimates
        n_shares = np.mean(n_shares_estimates) if n_shares_estimates else 1.0
        
        return {
            'strategy': 'collar',
            'stock_shares': n_shares,
            'put_strike': K1,
            'call_strike': K2,
            'put_premium': P,
            'call_premium': C,
            'stock_investment': n_shares * self.S_0,
            'put_cost': n_shares * P,
            'call_proceeds': n_shares * C,
            'net_cost': n_shares * (self.S_0 + P - C),
            'payoff_check': [self._check_collar_payoff(S_T, K1, K2, n_shares, P, C) 
                           for S_T in S_T_scenarios]
        }
    
    def complete_collar_given_put_call_strikes(self, K1, K2, target_cost=0):
        """
        Given put strike (K1) and call strike (K2), find stock position for target net cost
        
        Parameters:
        K1 (float): Put strike price - the protective floor level
                    Example: K1=95 protects against losses below $95
        K2 (float): Call strike price - the profit ceiling level  
                    Example: K2=105 caps gains above $105
        target_cost (float): Desired net cost of the entire collar strategy
                            Default: 0 (zero-cost collar)
                            Example: target_cost=5000 means willing to pay $5k net
        
        Returns:
        Dictionary with required stock shares and cost analysis
        """
        P = self.get_option_price(K1, 'put')
        C = self.get_option_price(K2, 'call')
        
        # Net cost = n_shares * (S_0 + P - C) = target_cost
        # Solve for n_shares
        cost_per_share = self.S_0 + P - C
        
        if abs(cost_per_share) < 1e-10:
            n_shares = float('inf')  # Zero-cost collar
        else:
            n_shares = target_cost / cost_per_share
        
        return {
            'strategy': 'collar',
            'stock_shares': n_shares,
            'put_strike': K1,
            'call_strike': K2,
            'put_premium': P,
            'call_premium': C,
            'cost_per_share': cost_per_share,
            'net_cost': target_cost,
            'zero_cost_collar': abs(cost_per_share) < 1e-10
        }
    
    def complete_collar_find_call_strike(self, K1, stock_shares=1, target_cost=0):
        """
        Given put strike (K1) and stock position, find call strike (K2) for target cost
        
        Parameters:
        K1 (float): Put strike price - the downside protection level
                    Example: K1=90 provides protection below $90
        stock_shares (float): Number of stock shares in the position
                             Default: 1 share
                             Example: stock_shares=1000 for 1,000 shares
                             Can be fractional for partial hedging
        target_cost (float): Desired net cost of the collar strategy
                            Default: 0 (zero-cost collar)
                            Positive: willing to pay this amount
                            Negative: want to receive this net credit
        
        Returns:
        Dictionary with optimal call strike and strategy details
        """
        P = self.get_option_price(K1, 'put')
        
        # target_cost = stock_shares * (S_0 + P - C)
        # Solve for C (call premium)
        required_call_premium = (stock_shares * (self.S_0 + P) - target_cost) / stock_shares
        
        # Find strike that gives this premium
        def objective(K2):
            C = self.get_option_price(K2, 'call')
            return (C - required_call_premium)**2
        
        try:
            # Search in reasonable range
            result = minimize_scalar(objective, bounds=(self.S_0*0.8, self.S_0*2.0), method='bounded')
            K2_optimal = result.x
            C_optimal = self.get_option_price(K2_optimal, 'call')
            
            return {
                'strategy': 'collar',
                'stock_shares': stock_shares,
                'put_strike': K1,
                'call_strike': K2_optimal,
                'put_premium': P,
                'call_premium': C_optimal,
                'net_cost': target_cost,
                'optimization_success': result.success,
                'actual_call_premium': C_optimal,
                'required_call_premium': required_call_premium
            }
        except:
            return {
                'error': 'Could not find suitable call strike',
                'required_call_premium': required_call_premium,
                'put_strike': K1,
                'put_premium': P
            }
    
    def complete_collar_find_put_strike(self, K2, stock_shares=1, target_cost=0):
        """
        Given call strike (K2) and stock position, find put strike (K1) for target cost
        
        Parameters:
        K2 (float): Call strike price - the upside cap level
                    Example: K2=110 means you give up gains above $110
        stock_shares (float): Number of stock shares being hedged
                             Default: 1 share
                             Example: stock_shares=500 to hedge 500 shares
        target_cost (float): Desired net cost of the collar strategy
                            Default: 0 (zero-cost collar)
                            Example: target_cost=-1000 means want $1k net credit
        
        Returns:
        Dictionary with optimal put strike and strategy details
        """
        C = self.get_option_price(K2, 'call')
        
        # target_cost = stock_shares * (S_0 + P - C)
        # Solve for P (put premium)
        required_put_premium = (target_cost - stock_shares * (self.S_0 - C)) / stock_shares
        
        # Find strike that gives this premium
        def objective(K1):
            P = self.get_option_price(K1, 'put')
            return (P - required_put_premium)**2
        
        try:
            # Search in reasonable range
            result = minimize_scalar(objective, bounds=(self.S_0*0.3, self.S_0*1.2), method='bounded')
            K1_optimal = result.x
            P_optimal = self.get_option_price(K1_optimal, 'put')
            
            return {
                'strategy': 'collar',
                'stock_shares': stock_shares,
                'put_strike': K1_optimal,
                'call_strike': K2,
                'put_premium': P_optimal,
                'call_premium': C,
                'net_cost': target_cost,
                'optimization_success': result.success,
                'actual_put_premium': P_optimal,
                'required_put_premium': required_put_premium
            }
        except:
            return {
                'error': 'Could not find suitable put strike',
                'required_put_premium': required_put_premium,
                'call_strike': K2,
                'call_premium': C
            }
    
    # THREE-WAY COLLAR COMPLETION METHODS
    def complete_three_way_given_three_strikes(self, K1, K2, K3, target_cost=0):
        """
        Given all three strikes, calculate required stock position for target cost
        
        Parameters:
        K1 (float): Short put strike - the lowest strike where you SELL protection
                    Example: K1=85 means you sell puts at $85 (collect premium but take risk below $85)
        K2 (float): Long put strike - the middle strike where you BUY protection  
                    Example: K2=95 means you buy puts at $95 (pay premium for protection below $95)
                    Note: K2 > K1 (you're protected between K1 and K2)
        K3 (float): Short call strike - the highest strike where you SELL calls
                    Example: K3=110 means you sell calls at $110 (collect premium but cap gains above $110)
                    Note: K3 > K2 > K1
        target_cost (float): Desired net cost of the entire 3-way collar strategy
                            Default: 0 (zero-cost strategy)
                            Example: target_cost=2000 means willing to pay $2k net
        
        Returns:
        Dictionary with required stock shares and comprehensive strategy details
        """
        P1 = self.get_option_price(K1, 'put')  # Short put
        P2 = self.get_option_price(K2, 'put')  # Long put
        C = self.get_option_price(K3, 'call')  # Short call
        
        # Net cost = n_shares * (S_0 + P2 - P1 - C) = target_cost
        cost_per_share = self.S_0 + P2 - P1 - C
        
        if abs(cost_per_share) < 1e-10:
            n_shares = float('inf')  # Zero-cost strategy
        else:
            n_shares = target_cost / cost_per_share
        
        return {
            'strategy': '3-way collar',
            'stock_shares': n_shares,
            'short_put_strike': K1,
            'long_put_strike': K2,
            'short_call_strike': K3,
            'short_put_premium': P1,
            'long_put_premium': P2,
            'call_premium': C,
            'cost_per_share': cost_per_share,
            'net_cost': target_cost,
            'zero_cost_strategy': abs(cost_per_share) < 1e-10
        }
    
    def complete_three_way_find_call_strike(self, K1, K2, stock_shares=1, target_cost=0):
        """
        Given K1, K2 and stock position, find K3 (call strike) for target cost
        
        Parameters:
        K1 (float): Short put strike - where you sell put protection
                    Example: K1=80 means you sell puts at $80 (take on risk below $80)
        K2 (float): Long put strike - where you buy put protection
                    Example: K2=90 means you buy puts at $90 (protected below $90)
                    Note: Must be > K1
        stock_shares (float): Number of stock shares in the position
                             Default: 1 share
                             Example: stock_shares=2000 for 2,000 shares
        target_cost (float): Desired net cost of the strategy
                            Default: 0 (zero-cost strategy)
                            Negative values mean you want net credit
        
        Returns:
        Dictionary with optimal call strike (K3) and strategy details
        """
        P1 = self.get_option_price(K1, 'put')
        P2 = self.get_option_price(K2, 'put')
        
        # target_cost = stock_shares * (S_0 + P2 - P1 - C)
        required_call_premium = (stock_shares * (self.S_0 + P2 - P1) - target_cost) / stock_shares
        
        def objective(K3):
            C = self.get_option_price(K3, 'call')
            return (C - required_call_premium)**2
        
        try:
            result = minimize_scalar(objective, bounds=(self.S_0*0.8, self.S_0*2.5), method='bounded')
            K3_optimal = result.x
            C_optimal = self.get_option_price(K3_optimal, 'call')
            
            return {
                'strategy': '3-way collar',
                'stock_shares': stock_shares,
                'short_put_strike': K1,
                'long_put_strike': K2,
                'short_call_strike': K3_optimal,
                'short_put_premium': P1,
                'long_put_premium': P2,
                'call_premium': C_optimal,
                'net_cost': target_cost,
                'optimization_success': result.success
            }
        except:
            return {
                'error': 'Could not find suitable call strike',
                'required_call_premium': required_call_premium
            }
    
    def complete_three_way_find_long_put_strike(self, K1, K3, stock_shares=1, target_cost=0):
        """
        Given K1, K3 and stock position, find K2 (long put strike) for target cost
        
        Parameters:
        K1 (float): Short put strike - the lowest level where you sell puts
                    Example: K1=75 means you sell puts at $75 (take risk below $75)
        K3 (float): Short call strike - the highest level where you sell calls  
                    Example: K3=115 means you sell calls at $115 (cap gains above $115)
        stock_shares (float): Number of stock shares being hedged
                             Default: 1 share
                             Example: stock_shares=800 for 800 shares
        target_cost (float): Desired net cost of the strategy
                            Default: 0 (zero-cost strategy)
                            Example: target_cost=1500 means willing to pay $1.5k net
        
        Returns:
        Dictionary with optimal long put strike (K2) and strategy details
        Note: K2 will be between K1 and K3, typically closer to current stock price
        """
        P1 = self.get_option_price(K1, 'put')
        C = self.get_option_price(K3, 'call')
        
        # target_cost = stock_shares * (S_0 + P2 - P1 - C)
        required_put_premium = (target_cost - stock_shares * (self.S_0 - P1 - C)) / stock_shares
        
        def objective(K2):
            P2 = self.get_option_price(K2, 'put')
            return (P2 - required_put_premium)**2
        
        try:
            # K2 should be between K1 and current price typically
            result = minimize_scalar(objective, bounds=(K1, self.S_0*1.1), method='bounded')
            K2_optimal = result.x
            P2_optimal = self.get_option_price(K2_optimal, 'put')
            
            return {
                'strategy': '3-way collar',
                'stock_shares': stock_shares,
                'short_put_strike': K1,
                'long_put_strike': K2_optimal,
                'short_call_strike': K3,
                'short_put_premium': P1,
                'long_put_premium': P2_optimal,
                'call_premium': C,
                'net_cost': target_cost,
                'optimization_success': result.success
            }
        except:
            return {
                'error': 'Could not find suitable long put strike',
                'required_put_premium': required_put_premium
            }
    
    def complete_three_way_find_short_put_strike(self, K2, K3, stock_shares=1, target_cost=0):
        """
        Given K2, K3 and stock position, find K1 (short put strike) for target cost
        
        Parameters:
        K2 (float): Long put strike - where you buy protective puts
                    Example: K2=90 means you buy puts at $90 for downside protection
        K3 (float): Short call strike - where you sell calls for income
                    Example: K3=120 means you sell calls at $120 (cap upside at $120)
        stock_shares (float): Number of stock shares in the position
                             Default: 1 share  
                             Example: stock_shares=1500 for 1,500 shares
        target_cost (float): Desired net cost of the entire strategy
                            Default: 0 (zero-cost strategy)
                            Example: target_cost=-500 means want $500 net credit
        
        Returns:
        Dictionary with optimal short put strike (K1) and strategy details
        Note: K1 will be < K2, representing the level below which you take on additional risk
        """
        P2 = self.get_option_price(K2, 'put')
        C = self.get_option_price(K3, 'call')
        
        # target_cost = stock_shares * (S_0 + P2 - P1 - C)
        required_short_put_premium = (stock_shares * (self.S_0 + P2 - C) - target_cost) / stock_shares
        
        def objective(K1):
            P1 = self.get_option_price(K1, 'put')
            return (P1 - required_short_put_premium)**2
        
        try:
            # K1 should be below K2
            result = minimize_scalar(objective, bounds=(self.S_0*0.3, K2*0.99), method='bounded')
            K1_optimal = result.x
            P1_optimal = self.get_option_price(K1_optimal, 'put')
            
            return {
                'strategy': '3-way collar',
                'stock_shares': stock_shares,
                'short_put_strike': K1_optimal,
                'long_put_strike': K2,
                'short_call_strike': K3,
                'short_put_premium': P1_optimal,
                'long_put_premium': P2,
                'call_premium': C,
                'net_cost': target_cost,
                'optimization_success': result.success
            }
        except:
            return {
                'error': 'Could not find suitable short put strike',
                'required_short_put_premium': required_short_put_premium
            }
    
    # PORTFOLIO CALCULATION METHODS
    def calculate_portfolio_weights(self, assets_info, target_hedge_ratio=1.0):
        """
        Calculate portfolio weights for hedging and analyze hedge effectiveness
        
        Parameters:
        assets_info (dict): Dictionary containing information about each asset in the portfolio
                           Format: {
                               'asset_name': {
                                   'price': float,      # Current market price per unit
                                   'quantity': float,   # Number of units held (+/- for long/short)
                                   'beta': float,       # For stocks: sensitivity to market (optional)
                                   'delta': float,      # For options: price sensitivity to underlying
                                   'premium': float,    # For options: option price per contract
                                   'strike': float      # For options: strike price (optional, for reference)
                               }
                           }
                           
        Example assets_info:
        {
            'stock': {
                'price': 100,        # Stock trading at $100
                'quantity': 1000,    # Long 1,000 shares  
                'beta': 1.0          # Moves 1:1 with market
            },
            'protective_put': {
                'strike': 95,        # Put with $95 strike
                'premium': 3.2,      # Costs $3.20 per share
                'quantity': 10,      # 10 contracts (1,000 shares worth)
                'delta': -0.3        # Put delta (negative for puts)
            },
            'covered_call': {
                'strike': 105,       # Call with $105 strike  
                'premium': 2.1,      # Receive $2.10 per share
                'quantity': -10,     # Short 10 contracts (negative = sold)
                'delta': 0.7         # Call delta (positive for calls)
            }
        }
        
        target_hedge_ratio (float): Desired hedging effectiveness ratio
                                   Default: 1.0 (fully hedged)
                                   0.5 = 50% hedged, 2.0 = 200% hedged
        
        Returns:
        Dictionary with portfolio analysis including:
        - Individual asset values and portfolio weights
        - Total portfolio value and delta exposure
        - Hedge effectiveness metrics
        - Delta neutrality assessment
        """
        total_portfolio_value = 0
        asset_values = {}
        
        # Calculate individual asset values
        for asset_name, info in assets_info.items():
            if asset_name == 'stock':
                value = info['price'] * info['quantity']
            else:  # Options
                value = info['premium'] * info.get('quantity', 100)  # Default 100 contracts
            
            asset_values[asset_name] = value
            total_portfolio_value += value
        
        # Calculate weights
        weights = {}
        for asset_name, value in asset_values.items():
            weights[asset_name] = value / total_portfolio_value
        
        # Calculate hedge effectiveness
        portfolio_delta = 0
        for asset_name, info in assets_info.items():
            if asset_name == 'stock':
                delta = 1.0  # Stock delta is 1
                quantity = info['quantity']
            else:
                delta = info.get('delta', 0)
                quantity = info.get('quantity', 100)
            
            portfolio_delta += delta * quantity
        
        return {
            'asset_values': asset_values,
            'total_value': total_portfolio_value,
            'weights': weights,
            'portfolio_delta': portfolio_delta,
            'hedge_ratio': abs(portfolio_delta) / assets_info.get('stock', {}).get('quantity', 1),
            'is_delta_neutral': abs(portfolio_delta) < 0.01 * total_portfolio_value
        }
    
    def _check_collar_payoff(self, S_T, K1, K2, n_shares, P, C):
        """Helper method to check collar payoff"""
        payoff = n_shares * (S_T + max(0, K1 - S_T) - max(0, S_T - K2))
        return {'stock_price': S_T, 'payoff': payoff}

In [7]:
def example_collar_completion():
    """Examples of collar completion"""
    print("=== COLLAR COMPLETION EXAMPLES ===\n")
    
    # Initialize
    completer = HedgingCompleter(S_0=100, r=0.05, t=0.25, sigma=0.25)
    
    # Example 1: Given both strikes, find stock position
    print("1. Given put strike ($95) and call strike ($105), find stock position:")
    result1 = completer.complete_collar_given_strikes(K1=95, K2=105, target_payoff=100)
    print(f"   Stock shares needed: {result1['stock_shares']:.2f}")
    print(f"   Net cost: ${result1['net_cost']:.2f}")
    print()
    
    # Example 2: Given put strike, find call strike for zero cost
    print("2. Given put strike ($95), find call strike for zero-cost collar:")
    result2 = completer.complete_collar_find_call_strike(K1=95, target_cost=0)
    if 'error' not in result2:
        print(f"   Call strike needed: ${result2['call_strike']:.2f}")
        print(f"   Zero cost achieved: {result2.get('zero_cost_collar', False)}")
    else:
        print(f"   {result2['error']}")
    print()
    
    # Example 3: Given call strike, find put strike for specific cost
    print("3. Given call strike ($105), find put strike for $2000 net cost:")
    result3 = completer.complete_collar_find_put_strike(K2=105, stock_shares=100, target_cost=2000)
    if 'error' not in result3:
        print(f"   Put strike needed: ${result3['put_strike']:.2f}")
        print(f"   Actual net cost: ${result3['net_cost']:.2f}")
    else:
        print(f"   {result3['error']}")
    print()

example_collar_completion()


=== COLLAR COMPLETION EXAMPLES ===

1. Given put strike ($95) and call strike ($105), find stock position:
   Stock shares needed: 1.00
   Net cost: $99.08

2. Given put strike ($95), find call strike for zero-cost collar:
   Call strike needed: $80.00
   Zero cost achieved: False

3. Given call strike ($105), find put strike for $2000 net cost:
   Put strike needed: $37.89
   Actual net cost: $2000.00



In [8]:
def example_three_way_collar_completion():
    """Examples of 3-way collar completion"""
    print("=== 3-WAY COLLAR COMPLETION EXAMPLES ===\n")
    
    completer = HedgingCompleter(S_0=100, r=0.05, t=0.25, sigma=0.25)
    
    # Example 1: Given all strikes, find stock position for zero cost
    print("1. Given all strikes (K1=$85, K2=$95, K3=$110), find stock position:")
    result1 = completer.complete_three_way_given_three_strikes(K1=85, K2=95, K3=110, target_cost=0)
    print(f"   Stock shares needed: {result1['stock_shares']:.2f}")
    print(f"   Cost per share: ${result1['cost_per_share']:.2f}")
    print()
    
    # Example 2: Given K1 and K2, find K3 for zero cost
    print("2. Given K1=$85 and K2=$95, find K3 for zero-cost strategy:")
    result2 = completer.complete_three_way_find_call_strike(K1=85, K2=95, target_cost=0)
    if 'error' not in result2:
        print(f"   Call strike needed: ${result2['short_call_strike']:.2f}")
    else:
        print(f"   {result2['error']}")
    print()

example_three_way_collar_completion()

=== 3-WAY COLLAR COMPLETION EXAMPLES ===

1. Given all strikes (K1=$85, K2=$95, K3=$110), find stock position:
   Stock shares needed: 0.00
   Cost per share: $99.95

2. Given K1=$85 and K2=$95, find K3 for zero-cost strategy:
   Call strike needed: $80.00



In [9]:
def example_portfolio_calculation():
    """Example of portfolio weight calculation"""
    print("=== PORTFOLIO CALCULATION EXAMPLE ===\n")
    
    completer = HedgingCompleter(S_0=100, r=0.05, t=0.25, sigma=0.25)
    
    # Define portfolio
    assets = {
        'stock': {'price': 100, 'quantity': 1000, 'beta': 1.0},
        'put_95': {'strike': 95, 'premium': 3.2, 'quantity': 10, 'delta': -0.3},
        'call_105': {'strike': 105, 'premium': 2.1, 'quantity': -5, 'delta': 0.7}  # Short calls
    }
    
    result = completer.calculate_portfolio_weights(assets)
    
    print("Portfolio Analysis:")
    print(f"Total Portfolio Value: ${result['total_value']:,.2f}")
    print(f"Portfolio Delta: {result['portfolio_delta']:.2f}")
    print(f"Delta Neutral: {result['is_delta_neutral']}")
    print("\nAsset Weights:")
    for asset, weight in result['weights'].items():
        print(f"  {asset}: {weight:.1%} (${result['asset_values'][asset]:,.2f})")


example_portfolio_calculation()

=== PORTFOLIO CALCULATION EXAMPLE ===

Portfolio Analysis:
Total Portfolio Value: $100,021.50
Portfolio Delta: 993.50
Delta Neutral: True

Asset Weights:
  stock: 100.0% ($100,000.00)
  put_95: 0.0% ($32.00)
  call_105: -0.0% ($-10.50)
